# Experiment: selekčný tlak

Cieľom je merať selekčný tlak rôznych metód selekcie. Ide o to, ako veľmi jednotlivé selekčné metódy uprednostňujú jedincov s vyššou vhodnosťou oproti jedincom s vhodnosťou nižšou. 

Selekčný tlak bude meraný nepriamo pomocou určenia **doby prevzatia** (takeover time), čo je doba ktorú potrebuje jedinec s najväčšou vhodnosťou na to, aby sa stal dominantným v populácii (aby vyplnil celú populáciu svojimi kópiami). Čím je doba prevzatia menšia, tým silnejšie musel byť najlepší jedinec preferovaný voči ostatným jedincom a teda tým je vyvinutý selekčný tlak väčší.

## Podstata experimentu

Je daná skupina jedincov, pričom každý z nich má pridelenú vhodnosť. Existuje jeden jedinec s najväčšou vhodnosťou (trebárs Φ(best)=1.0), zatiaľ čo ostatné jedince majú vhodnosť menšiu (môžu mať navzájom rôzne vhodnosti ale nie je to nutné).

Realizuje sa selekčný proces ako séria selekcií tvaru P -> P' -> P'' -> ... -> P''''''', pričom každá selekcia robí výber zo skupiny jedincov, ktorá vznikla ako výsledok predchádzajúcej selekcie. Toto reťazenie pokračuje dovtedy, kým nenastane jedna zo situácií:
1. najlepší jedinec úplne vypadne z populácie (nebude v nej ani jedna jeho kópia),
2. najlepší jedinec vyplní svojimi kópiami celú populáciu,
3. uplynie maximálne povolený počet selekcií v sekvencii.

Keďže sa jedná o stochastický proces, tak musí byť viackrát opakovaný pre získanie spoľahlivej vzorky dát. Ak v rámci tohto opakovania sa viackrát stalo, že najlepší jedinec dominoval v populácii, tak sa určí priemerný počet selekcií, ktorý je potrebným pre dosiahnutie tejto dominancie.

## Príprava
Pre prípravu experimentu je potrebné doplniť (nahradiť) kód bunke venovanej selekčnej metóde. Aktuálne je v nej imlementované vzorkovanie SSwR (teda základná ruleta). Je možné meniť metódu pre:
1. premapovanie vhodnosti (aktuálne identita t.j. žiadne)
2. určenie pravdepodobnosti selekcie (aktuálne proporcionálna selekcia)
3. vzorkovanie jedného rodiča (aktuálne SSwR)
4. selekciu rodičov (aktuálne opakovaná selekcia po jednom rodičovi)

In [70]:
# zadanie: premapovanie exponenecialnym zotriedenim,vypocet pravdepodobnosti proporcionalnou selekciou , ruleta SuS- task 2
import random
from functools import reduce
from operator import add
import math

In [71]:
# Generovanie populácie

μ = 100  # veľkosť populácie
jedince = [1] + [0 for i in range(μ-1)]
#jedince

In [72]:

MIN = 0.2  # minimalna mozna vhodnosť jedincov (okrem najlepsieho)
MAX = 0.5  # maximalna mozna vhodnosť jedincov (okrem najlepsieho)

def Φ(jedinec):
    if jedinec == 1:
        return 1.0
    else:
        return random.uniform(MIN, MAX)

populacia = [(i, Φ(i)) for i in jedince]

In [86]:
# Selekčná metóda - vyberá sa toľko rodičov, koľko je jedincov v populácii

def exponential_fitness_remapping(populacia, c=0.98):
    sorted_populacia = sorted(populacia, key=lambda x: x[1], reverse=True)
    remapped_populacia = [(indiv[0], c ** (rank + 1)) for rank, indiv in enumerate(sorted_populacia)]
    return remapped_populacia

def urcenie_pravdepodobnosti_selekcie(remapped_populacia):
    premap_Φ = [indiv[1] for indiv in remapped_populacia]
    sum_Φ = reduce(add, premap_Φ)
    ps = [Φ / sum_Φ for Φ in premap_Φ]
    return [(remapped_populacia[i][0], ps[i]) for i in range(len(remapped_populacia))]


def stochastic_universal_sampling(cumulative_probs, population_size):
    pointer_spacing = 1.0 / population_size
    start_point = random.uniform(0, pointer_spacing)
    pointers = [start_point + i * pointer_spacing for i in range(population_size)]
    
    selected_individuals = []
    current_member_idx = 0
    for pointer in pointers:
        while cumulative_probs[current_member_idx][1] < pointer:
            current_member_idx += 1
        selected_individuals.append(cumulative_probs[current_member_idx][0])
    
    return selected_individuals


# Apply exponential fitness remapping to the population
remapped_populacia = exponential_fitness_remapping(populacia)

# Determine proportional selection probabilities
proportional_selection = urcenie_pravdepodobnosti_selekcie(remapped_populacia)

# Convert to cumulative probabilities
cumulative_probabilities = []
cumulative_sum = 0
for individual, probability in proportional_selection:
    cumulative_sum += probability
    cumulative_probabilities.append((individual, cumulative_sum))

# Adjust the last cumulative probability for normalization
cumulative_probabilities[-1] = (cumulative_probabilities[-1][0], 1)

# Perform Stochastic Universal Sampling to select parents
parents = stochastic_universal_sampling(cumulative_probabilities, μ)

# Print selected parents for the next generation
print(parents)


[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [87]:
# Experiment - jeden beh

MAXPOC_SELEKCII = 2 * μ
print(populacia)

def seria_selekcii(populacia):
    id_of_best_individual = 1  # Assuming the best individual has ID 1
    vyber_z_populacie = populacia
    for i in range(MAXPOC_SELEKCII):
        remapped_populacia = exponential_fitness_remapping(vyber_z_populacie)
        proportional_selection = urcenie_pravdepodobnosti_selekcie(remapped_populacia)
        
        # Convert to cumulative probabilities
        cumulative_probabilities = []
        cumulative_sum = 0
        for individual, probability in proportional_selection:
            cumulative_sum += probability
            cumulative_probabilities.append((individual, cumulative_sum))
        
        # Adjust the last cumulative probability for normalization
        if len(cumulative_probabilities) > 0 and cumulative_probabilities[-1][1] < 1:
            cumulative_probabilities[-1] = (cumulative_probabilities[-1][0], 1)
        
        # Select individuals using SUS
        selected_individual_ids = stochastic_universal_sampling(cumulative_probabilities, μ)
        
        # Count the occurrences of the best individual in the selected IDs
        poc_kopii_najlepsieho = selected_individual_ids.count(id_of_best_individual)
        
        if poc_kopii_najlepsieho == μ:
            return 'vyplnenie', i
        if poc_kopii_najlepsieho == 0:
            return 'strata', i+1
        
    return 'vyskyt', i+1


[(1, 1.0), (0, 0.22767923611155033), (0, 0.2733235783007439), (0, 0.21845040341328095), (0, 0.4972381534890299), (0, 0.32378290616076805), (0, 0.3121212540168811), (0, 0.27770755026674737), (0, 0.45503845450164443), (0, 0.26419141426105686), (0, 0.4058213394031148), (0, 0.47548840247508506), (0, 0.3886994946111753), (0, 0.3842251686865883), (0, 0.3885312325552678), (0, 0.46238918236582194), (0, 0.3131895280352735), (0, 0.4495427597567535), (0, 0.38683622606019885), (0, 0.44758345440376107), (0, 0.32602932495637593), (0, 0.2652672965741692), (0, 0.374314572981744), (0, 0.20885334142708492), (0, 0.2071855036928773), (0, 0.301287704090305), (0, 0.35424132251539486), (0, 0.23640343924803758), (0, 0.40688513896184), (0, 0.2914330394048579), (0, 0.49826843139747645), (0, 0.2919530063237046), (0, 0.4742151122697109), (0, 0.4065082142796448), (0, 0.4787140713338406), (0, 0.4910672213491287), (0, 0.2608693377803451), (0, 0.4629379712897952), (0, 0.4911906168194428), (0, 0.2000027844950292), (0,

In [88]:
# Experiment - opakovane sekvencie selekcii

OPAKOVANIA = 1000

vysledky = [seria_selekcii(populacia) for _ in range(OPAKOVANIA)]

print(
    f"Pocet behov "
    f"\n   so stratou najlepsieho    : {len(list(filter(lambda x: x[0] == 'strata', vysledky)))}"
    f"\n   s dominanciou najlepsieho : {len(list(filter(lambda x: x[0] == 'vyplnenie', vysledky)))}"
    f"\n   s vyskytom najlepsieho    : {len(list(filter(lambda x: x[0] == 'vyskyt', vysledky)))}"
)

vyplnenia = list(map(lambda x: x[1], (filter(lambda x: x[0] == 'vyplnenie', vysledky))))
if vyplnenia != []:
    print(f"Pri dominancii najlepsi dominoval priemerne v {reduce(add,vyplnenia)/len(vyplnenia)}. iteracii")

Pocet behov 
   so stratou najlepsieho    : 0
   s dominanciou najlepsieho : 0
   s vyskytom najlepsieho    : 1000
